# Tutorial: Coding a Process App that uses several data sets from a ZIP file
This tutorial is using the simple Procure to Pay data set that could be extracted from systems like SAP.

Instead of connecting the process app to an external system, we leverage data tables that have been extracted previously and that are going to be transformed to create an event log for IBM Process Mining.

The full source code of the python code used to create this app is here: [P2P_data_xform_lab.py](./P2P_data_xform_lab.py).

## Data Sources
When launching the process app, the user is asked to upload a ZIP file that contains the following files:
- requisition.csv
- procurement.csv
- invoice.csv

requisition.csv contains a row for each requisition uniquely identified with a Req_ID. Along with many attributes, two series of columns are interesting as they store important milestones of the requisition: the creation date and the release date. We will create a process mining event (activity) for the creation and for the release of each requisition.

procurement.csv is a ready-to-use event log for process mining that includes all the P2P events except the requisitions. Each procurement case includes several events such as 'Order Item Created', 'Invoice Registered', and many more.  

invoice.csv contains information about each invoice that we would like to add into each procurement process that matches the Invoice_ID.

## Transformation Process
In this process app python code, we are going to:
- Extract the 3 csv files from the ZIP file uploaded by the user
- Transform requisition.csv into a process mining event log
- Keep procurement.csv that is a ready-to-use event log for process mining
- Merge (left join) invoice.csv with procurement.csv to add invoice information that is missing from procurement.csv
- Concatenate requisition and procurement events to form a multilevel process (2 columns used as case IDs: Req_ID and Proc_ID)

## Python Code Template
IBM Process Mining will call the ```execute(context)``` function when starting the data transformation of the process app. This is where we are doing most of the data transformation work.

The context typically contains the user input parameters, and the name of the ZIP file that the user uploaded.

To run this program on you local machine with this Jupyter Notebook, you need to install pandas and zipfile:
```shell
pip install pandas
``````

In [38]:
import pandas as pd
from zipfile import ZipFile
import time

def execute(context):
    # Unzip the file that includes the 3 files that we need to transform and merge
    myFileUploadName = context["fileUploadName"]
    with ZipFile(myFileUploadName, 'r') as f:
        f.extractall()


context = {'fileUploadName':'P2P.zip'}
execute(context)

When you execute the code above, you should see that you now have the 3 CSV files extracted in your local directory. 
IBM Process Mining executes this code from a directory created specifically for the process app, that contains the python file and the uploaded ZIP file. This is where the 3 files are extracted.

## Using Pandas to create and manipulate data tables
Pandas is a powerful table data manipulation library used by thousands of data engineers: https://pandas.pydata.org/docs/.
We are going to use Pandas dataframes to store and manipulate the data loaded from the CSV files.

At the end of the ```execute()``` function, we need to return a Pandas dataframe that contains the event logs.

## Start Simple: Load the procurement file
Note that we wait a bit that the files are extracted before loading them as Pandas dataframes.
Then we just need to call the Pandas ```read_csv()``` method that loads the procurement event log into a dataframe.
At that stage, that's the dataframe that the ```execute()``` function returns for the process app, and that could be good enough if we wanted to just mine the procurement process.

In [39]:
import pandas as pd
from zipfile import ZipFile
import time

def execute(context):
    # Unzip the file that includes the 3 files that we need to transform and merge
    myFileUploadName = context["fileUploadName"]
    with ZipFile(myFileUploadName, 'r') as f:
        f.extractall()

    time.sleep(5)
    procurements = pd.read_csv('./procurement.csv', low_memory=False)
    return procurements


context = {'fileUploadName':'P2P.zip'}
procurements = execute(context)

## Experiment a few useful Pandas methods.
Experiment a few common pandas methods on dataframes. The code cell below displays the result of the last code line. You can comment or move the lines.
This is just to show a abstract of how we can handle tables with Pandas, with methods that are often used in our data transformation process.
You are stronghly encouraged to study https://pandas.pydata.org/docs/index.html

In [54]:
procurements = pd.read_csv('./procurement.csv', low_memory=False)
procurements.columns # list of columns
procurements.describe() # statistics of the dataframe
procurements['Order_Vendor'] # Column of Order_Vendor
procurements['Order_Vendor'].unique() # List of Order_Vendor 
len(procurements['Order_Vendor'].unique()) # Number of distinct Order_Vendor
procurements['Order_Vendor'].isna() # test if 'Order_Vendor' is NaN
procurements[procurements['Order_Vendor'].isna()] # dataframe keeps procurements where 'Order_Vendor' is NaN
procurements[~procurements['Order_Vendor'].isna()] # dataframe keeps procurements where 'Order_Vendor' is not NaN
procurements['NEW_DIM_1'] = 'A VALUE' # adds a column with a value
procurements['NEW_DIM_2'] = procurements['Order_Vendor'] # set a column with the value of another column
procurements.rename(columns={'NEW_DIM_1':'DIM_3', 'NEW_DIM_2':'DIM_4'}) # rename columns in a copy of the df
# The next 3 commands are bound
procurements.rename(columns={'NEW_DIM_1':'DIM_3', 'NEW_DIM_2':'DIM_4'}, inplace=True) # rename columns in the original df
procurements['DIM_4'] # list column DIM_4
procurements.drop(columns=['DIM_3', 'DIM_4'], inplace=True) # Delete columns DIM_3 and DIM_4

procurements.iloc[1:4] # Select rows 1 to 4

# The next 3 commands are bound
# Creating a new column 'DIM_5' which value depends on a test in one column, and the value built from another
procurements.loc[procurements['Order_Vendor'].isna(),'DIM_5'] = 'Vendor unknown from ' + procurements['Purchase_Organization']
procurements.loc[~procurements['Order_Vendor'].isna(),'DIM_5'] = 'Vendor known from ' + procurements['Purchase_Organization']
procurements['DIM_5'].unique()

array(['Vendor known from IT10', 'Vendor known from IT11',
       'Vendor unknown from IT10', 'Vendor unknown from IT11', nan],
      dtype=object)

## Data transformation of the requisition.csv table
The requisition table stores each requisition in a single row that includes both the creation information and the release information. We create an event log that contains two activities: creation and release.


In [57]:
requisitions = pd.read_csv('./requisition.csv', low_memory=False)
requisitions.columns

Index(['Req_Header', 'Req_Line', 'Req_ID', 'Create_Date', 'Create_User',
       'Create_Role', 'Create_Type', 'Create_Source', 'Release_DateTime',
       'Release_User', 'Release_Role', 'Release_Type', 'Release_Source'],
      dtype='object')

### Requisition Created Activity
The create_requisitions event log is a copy of the requisitions dataframe. We rename some columns, and we drop the columns related to the release activity.

In [58]:
create_requisitions = requisitions.copy()
create_requisitions['activity'] = 'Requisition Created'
create_requisitions.rename(columns={'Create_Date': 'datetime', 'Create_User': 'user',
                        'Create_Role': 'role', 'Create_Type': 'type', 'Create_Source': 'source'}, inplace=True)
create_requisitions.drop(['Release_DateTime', 'Release_User', 'Release_Role',
                        'Release_Type', 'Release_Source'], axis=1, inplace=True)
create_requisitions

,Req_Header,Req_Line,Req_ID,datetime,user,role,type,source,activity
0,NaN,NaN,02a9abba-ca02-48c6-a77d-448d9a22bdd5,2019-01-09,USR01957,Job System,NaN,NaN,Requisition Created
1,NaN,NaN,057f6784-3b2d-4e4a-b5a5-5d27e12fbbbc,2019-01-09,USR01851,Job System,NaN,NaN,Requisition Created
2,NaN,NaN,0609f016-58e9-4d5e-8298-3d2c33fe6769,2019-01-16,ACQ04,Job System,NaN,NaN,Requisition Created
3,NaN,NaN,07a5950c-fdf4-4cb0-a54c-4ade6d10414b,2019-01-07,USR00245,Job System,NaN,NaN,Requisition Created
4,NaN,NaN,090ee711-f0ec-4a69-b202-e62e8a441812,2019-01-10,USR00193,Job System,NaN,NaN,Requisition Created
...,...,...,...,...,...,...,...,...,...
24128,90000137.0,30.0,0090000137_30,2018-11-28,MKT103,Secretary,Indirect,N.A.,Requisition Created
24129,90000137.0,40.0,0090000137_40,2018-11-28,MKT103,Secretary,Indirect,N.A.,Requisition Created
24130,90000137.0,50.0,0090000137_50,2018-11-28,MKT103,Secretary,Indirect,N.A.,Requisition Created
24131,90000137.0,60.0,0090000137_60,2018-11-28,MKT103,Secretary,Indirect,N.A.,Requisition Created


### Requisition Released Activity
The release_requisitions event log is also  a copy of the requisitions dataframe. We rename some columns, and we drop the columns related to the creation activity.

In [69]:
release_requisitions = requisitions.copy()
release_requisitions['activity'] = 'Requisition Released'
release_requisitions.rename(columns={'Release_DateTime': 'datetime', 'Release_User': 'user',
                        'Release_Role': 'role', 'Release_Type': 'type', 'Release_Source': 'source'}, inplace=True)
release_requisitions.drop(['Create_Date', 'Create_User', 'Create_Role',
                            'Create_Type', 'Create_Source'], axis=1, inplace=True)
# If some requisition cases are not complete, we remove rows where the requisition release date is NaN, as in
# this case, the activity did not yet occur. This is actually never the case.
release_requisitions = release_requisitions[release_requisitions['datetime'].notna()]
release_requisitions.columns

Index(['Req_Header', 'Req_Line', 'Req_ID', 'datetime', 'user', 'role', 'type',
       'source', 'activity'],
      dtype='object')

## Adding Invoice information to the procurement event log
The last transformation consists in appending the invoice details that are stored in the invoice.csv file, to the procurement event log.
In the procurement event log, we only have the Invoice_ID, and we have no information about the amount or the client.

In [66]:
procurements = pd.read_csv('./procurement.csv', low_memory=False)
procurements.columns # list of columns

Index(['Req_ID', 'PO_Header', 'PO_Line', 'PO_ID', 'MatDoc_Header',
       'MatDoc_Line', 'MatDoc_Year', 'MatDoc_ID', 'gr_h_y', 'Invoice_ID',
       'activity', 'datetime', 'user', 'role', 'rses_h', 'rses_l', 'rses_y',
       'mandt', 'bukrs', 'xblnr', 'fl_h', 'fl_y', 'value_old', 'value_new',
       'clear_doc', 'qmnum', 'data_gr_effettiva', 'usertype', 'Order_Type',
       'Purchasing_Group', 'Purch_Group_Type', 'Material_Group_Area',
       'Accounting_Type', 'Order_Vendor', 'Order_Source', 'Department',
       'Order_Amount', 'Material', 'lead_time_material', 'Material_Type',
       'Purch_Group_Area', 'Requisition_Plant', 'Order_Plant',
       'Material_Plant', 'data_gr_ordine', 'data_gr_stat', 'data_gr_ipo',
       'Paid_Amount', 'Paid_Vendor', 'split_ordine', 'split_riga_ordine',
       'missmatch_riga_oda', 'check_riga_gagm', 'consegna_ipotetica',
       'consegna_oda_ipotetica', '_consegna_statistica_ipotetica_',
       'pay_delay', 'pay_type', 'Req_Required_Vendor', 'Material_

The invoice table contains these invoice information:

In [67]:
invoices = pd.read_csv('./invoice.csv')
invoices.columns

Index(['Invoice_ID', 'Invoice_Header', 'Invoice_Year', 'Invoice_Amount',
       'Invoice_Vendor', 'Invoice_Due_Date', 'Invoice_Vendor_City'],
      dtype='object')

If we were using a database, we would join-left the two table using the Invoice_ID field. Pandas provides the same feature using the merge method.

In [68]:
procurements = procurements.merge(invoices, on="Invoice_ID", how="left")
procurements.columns

Index(['Req_ID', 'PO_Header', 'PO_Line', 'PO_ID', 'MatDoc_Header',
       'MatDoc_Line', 'MatDoc_Year', 'MatDoc_ID', 'gr_h_y', 'Invoice_ID',
       'activity', 'datetime', 'user', 'role', 'rses_h', 'rses_l', 'rses_y',
       'mandt', 'bukrs', 'xblnr', 'fl_h', 'fl_y', 'value_old', 'value_new',
       'clear_doc', 'qmnum', 'data_gr_effettiva', 'usertype', 'Order_Type',
       'Purchasing_Group', 'Purch_Group_Type', 'Material_Group_Area',
       'Accounting_Type', 'Order_Vendor', 'Order_Source', 'Department',
       'Order_Amount', 'Material', 'lead_time_material', 'Material_Type',
       'Purch_Group_Area', 'Requisition_Plant', 'Order_Plant',
       'Material_Plant', 'data_gr_ordine', 'data_gr_stat', 'data_gr_ipo',
       'Paid_Amount', 'Paid_Vendor', 'split_ordine', 'split_riga_ordine',
       'missmatch_riga_oda', 'check_riga_gagm', 'consegna_ipotetica',
       'consegna_oda_ipotetica', '_consegna_statistica_ipotetica_',
       'pay_delay', 'pay_type', 'Req_Required_Vendor', 'Material_

## Concatenating the dataframes to produce the final event log
The final step consists in concatenating the 2 requisition tables and the procurement table. 
Pandas concatenation is a union of the table columns. Therefore we end up with all the fields from each process, and we keep the common fields: 'datetime', 'user', 'role', 'type', 'source', 'activity'.

In [70]:
P2P_events = pd.concat([create_requisitions, release_requisitions, procurements])
P2P_events = P2P_events.convert_dtypes()  # applying the best known types
P2P_events.columns

Index(['Req_Header', 'Req_Line', 'Req_ID', 'datetime', 'user', 'role', 'type',
       'source', 'activity', 'PO_Header', 'PO_Line', 'PO_ID', 'MatDoc_Header',
       'MatDoc_Line', 'MatDoc_Year', 'MatDoc_ID', 'gr_h_y', 'Invoice_ID',
       'rses_h', 'rses_l', 'rses_y', 'mandt', 'bukrs', 'xblnr', 'fl_h', 'fl_y',
       'value_old', 'value_new', 'clear_doc', 'qmnum', 'data_gr_effettiva',
       'usertype', 'Order_Type', 'Purchasing_Group', 'Purch_Group_Type',
       'Material_Group_Area', 'Accounting_Type', 'Order_Vendor',
       'Order_Source', 'Department', 'Order_Amount', 'Material',
       'lead_time_material', 'Material_Type', 'Purch_Group_Area',
       'Requisition_Plant', 'Order_Plant', 'Material_Plant', 'data_gr_ordine',
       'data_gr_stat', 'data_gr_ipo', 'Paid_Amount', 'Paid_Vendor',
       'split_ordine', 'split_riga_ordine', 'missmatch_riga_oda',
       'check_riga_gagm', 'consegna_ipotetica', 'consegna_oda_ipotetica',
       '_consegna_statistica_ipotetica_', 'pay_delay', '

Eventually we reorder the columns to simplify the process mining mapping. To do so, copy the result of the columns method and reorder some.

In [73]:
P2P_events = P2P_events[['activity', 'datetime', 'user', 'role', 'type',
       'source', 'Req_ID','Req_Header', 'Req_Line', 'PO_Header', 'PO_Line', 'PO_ID', 'MatDoc_Header',
       'MatDoc_Line', 'MatDoc_Year', 'MatDoc_ID', 'gr_h_y', 'Invoice_ID',
       'rses_h', 'rses_l', 'rses_y', 'mandt', 'bukrs', 'xblnr', 'fl_h', 'fl_y',
       'value_old', 'value_new', 'clear_doc', 'qmnum', 'data_gr_effettiva',
       'usertype', 'Order_Type', 'Purchasing_Group', 'Purch_Group_Type',
       'Material_Group_Area', 'Accounting_Type', 'Order_Vendor',
       'Order_Source', 'Department', 'Order_Amount', 'Material',
       'lead_time_material', 'Material_Type', 'Purch_Group_Area',
       'Requisition_Plant', 'Order_Plant', 'Material_Plant', 'data_gr_ordine',
       'data_gr_stat', 'data_gr_ipo', 'Paid_Amount', 'Paid_Vendor',
       'split_ordine', 'split_riga_ordine', 'missmatch_riga_oda',
       'check_riga_gagm', 'consegna_ipotetica', 'consegna_oda_ipotetica',
       '_consegna_statistica_ipotetica_', 'pay_delay', 'pay_type',
       'Req_Required_Vendor', 'Material_Group', 'Invoice_Date',
       'Requisition_Vendor', 'Purchase_Organization', 'insert_date',
       'Invoice_Header', 'Invoice_Year', 'Invoice_Amount', 'Invoice_Vendor',
       'Invoice_Due_Date', 'Invoice_Vendor_City']]
P2P_events.columns

Index(['datetime', 'user', 'role', 'type', 'source', 'activity', 'Req_ID',
       'Req_Header', 'Req_Line', 'PO_Header', 'PO_Line', 'PO_ID',
       'MatDoc_Header', 'MatDoc_Line', 'MatDoc_Year', 'MatDoc_ID', 'gr_h_y',
       'Invoice_ID', 'rses_h', 'rses_l', 'rses_y', 'mandt', 'bukrs', 'xblnr',
       'fl_h', 'fl_y', 'value_old', 'value_new', 'clear_doc', 'qmnum',
       'data_gr_effettiva', 'usertype', 'Order_Type', 'Purchasing_Group',
       'Purch_Group_Type', 'Material_Group_Area', 'Accounting_Type',
       'Order_Vendor', 'Order_Source', 'Department', 'Order_Amount',
       'Material', 'lead_time_material', 'Material_Type', 'Purch_Group_Area',
       'Requisition_Plant', 'Order_Plant', 'Material_Plant', 'data_gr_ordine',
       'data_gr_stat', 'data_gr_ipo', 'Paid_Amount', 'Paid_Vendor',
       'split_ordine', 'split_riga_ordine', 'missmatch_riga_oda',
       'check_riga_gagm', 'consegna_ipotetica', 'consegna_oda_ipotetica',
       '_consegna_statistica_ipotetica_', 'pay_delay', '

Finally we can create a CSV file to test our event log in IBM Process Mining.
Note that we do not save the index (row numbers).

You can use this [backup file](./P2P%20LAB_2023-08-03_015726-0700.idp) to map some columns.

In [ ]:
P2P_events.to_csv('P2Peventlog.csv', index=None)

## Conclusion
The complete source code that we upload in the process app is shown in the next cell. You will need to upload an idp file that contains at least the column mapping. This [backup file](./P2P%20LAB_2023-08-03_015726-0700.idp) can be used to map some columns

In [ ]:
import pandas as pd
from zipfile import ZipFile
import time

def execute(context):

    # Unzip the file that includes the 3 files that we need to transform and merge
    myFileUploadName = context["fileUploadName"]
    with ZipFile(myFileUploadName, 'r') as f:
        f.extractall()

    # The 3 files are now stored in the Process Mining Server, in a directory dedicated to the process app, where the python code is stored and executed
    # as well as where the ZIP file loaded by the user is stored.
    # Therefore, all the unzipped files are accessible in the current directory
    # sleep a while until all files are extracted
    time.sleep(5)

    requisitions = pd.read_csv('./requisition.csv')
    create_requisitions = requisitions.copy()
    create_requisitions['activity'] = 'Requisition Created'
    create_requisitions.rename(columns={'Create_Date': 'datetime', 'Create_User': 'user',
                            'Create_Role': 'role', 'Create_Type': 'type', 'Create_Source': 'source'}, inplace=True)
    create_requisitions.drop(['Release_DateTime', 'Release_User', 'Release_Role',
                            'Release_Type', 'Release_Source'], axis=1, inplace=True)

    release_requisitions = requisitions.copy()
    release_requisitions['activity'] = 'Requisition Released'
    release_requisitions.rename(columns={'Release_DateTime': 'datetime', 'Release_User': 'user',
                        'Release_Role': 'role', 'Release_Type': 'type', 'Release_Source': 'source'}, inplace=True)
    release_requisitions.drop(['Create_Date', 'Create_User', 'Create_Role',
                            'Create_Type', 'Create_Source'], axis=1, inplace=True)
    # If some requisition cases are not complete, we remove rows where the requisition release date is NaN, as in
    # this case, the activity did not yet occur. This is actually never the case.
    release_requisitions = release_requisitions[release_requisitions['datetime'].notna()]

    # procurements
    procurements = pd.read_csv('./procurement.csv', low_memory=False)

    # invoices
    invoices = pd.read_csv('./invoice.csv')

    # Merging invoice.csv information into procurement.csv
    procurements = procurements.merge(invoices, on="Invoice_ID", how="left")

    # Finally we append the requisition and the procurement event logs to create the final event log. Again, we can remove the events with a null `datetime`P2P_events = pd.concat([P2P_events, procurement_events])
    P2P_events = pd.concat([create_requisitions, release_requisitions, procurements])
    # removing rows with no datetime if any
    P2P_events = P2P_events[P2P_events["datetime"].notna()]
    P2P_events = P2P_events.convert_dtypes()  # applying the best known types
    # Reordering columns to simplify mapping
    P2P_events = P2P_events[['activity','datetime', 'user', 'role', 'type',
       'source',  'Req_ID','Req_Header', 'Req_Line', 'PO_Header', 'PO_Line', 'PO_ID', 'MatDoc_Header',
       'MatDoc_Line', 'MatDoc_Year', 'MatDoc_ID', 'gr_h_y', 'Invoice_ID',
       'rses_h', 'rses_l', 'rses_y', 'mandt', 'bukrs', 'xblnr', 'fl_h', 'fl_y',
       'value_old', 'value_new', 'clear_doc', 'qmnum', 'data_gr_effettiva',
       'usertype', 'Order_Type', 'Purchasing_Group', 'Purch_Group_Type',
       'Material_Group_Area', 'Accounting_Type', 'Order_Vendor',
       'Order_Source', 'Department', 'Order_Amount', 'Material',
       'lead_time_material', 'Material_Type', 'Purch_Group_Area',
       'Requisition_Plant', 'Order_Plant', 'Material_Plant', 'data_gr_ordine',
       'data_gr_stat', 'data_gr_ipo', 'Paid_Amount', 'Paid_Vendor',
       'split_ordine', 'split_riga_ordine', 'missmatch_riga_oda',
       'check_riga_gagm', 'consegna_ipotetica', 'consegna_oda_ipotetica',
       '_consegna_statistica_ipotetica_', 'pay_delay', 'pay_type',
       'Req_Required_Vendor', 'Material_Group', 'Invoice_Date',
       'Requisition_Vendor', 'Purchase_Organization', 'insert_date',
       'Invoice_Header', 'Invoice_Year', 'Invoice_Amount', 'Invoice_Vendor',
       'Invoice_Due_Date', 'Invoice_Vendor_City']]

    return(P2P_events)


if __name__ == "__main__":

    context = {'fileUploadName':'P2P.zip'}
    df = execute(context)
    df.to_csv('P2Peventlog.csv', index=None)